<a href="https://colab.research.google.com/github/MilSur/ThorlabsPurchase/blob/main/ThorlabsPurchase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install selenium
!pip install bs4
!pip install docx
!pip install docxtpl
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from bs4 import BeautifulSoup
from docxtpl import DocxTemplate
from google.colab import files

############################################################
# PARAMETERS TO SET:
# please fill desired output filename, VAT, EUR/CZK exchange rate
# and most importantly product list with Thorlabs product name and quantity

output_name = '2021_January_Milos'

vat = 1.21
eur = 26.2

product_list = [
    ['KB1P/M', 2],
    ['RS3M', 2],
    ['RSH4/M', 2],
    ['PF175B', 2],
    ['RS100P4/M', 2],
    ['RS2P4M', 2],
    ['RS1M', 2],
    ['RS12P4/M', 2],
    ['KB25/M', 2],
    ['XRN25P/M', 2],
    ['AE4M6M', 10],
    ['BA1/M-P5', 3],
    ['BA1S/M-P5', 3],
    ['KB75/M', 4],
    ['KBT75/M', 2],
    ['LG11', 1],
    ['PH50/M-P5', 1],
    ['ST1XY-S/M', 1],
    ['TPS4', 2],
    ['TPSM1/M', 4],    
]

############################################################

# Docs:
thorlabs_template = 'thorlabs_temp.docx'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver', options=options)

products = []
for product in product_list:
    browser.get('https://www.thorlabs.com/thorproduct.cfm?partnumber='+str(product[0]))
    price_code = browser.find_elements_by_xpath("//tr/td/b[contains(text(), 'Price')]/../following-sibling::td")
    price = float(price_code[0].text[:-2].replace(',', '.'))
    title_code = browser.find_elements_by_xpath("//td[@class='PartTitle']")
    title = title_code[0].text
    title = title.replace(str(product[0])+' - ', '')
    products.append([product[0], title, product[1], price, '{:,}'.format(int(round(price*vat*eur*product[1], -1))).replace(',', ' ')])
browser.quit()
#print('Data successfully scraped!')

doc = DocxTemplate(thorlabs_template)

context = {
    'products': products,
    'vat': vat,
    'eur': eur,
    'total': int(sum([product[3]*vat*eur*product[2] for product in products]))/1000
}

doc.render(context)
doc.save(output_name+'.docx')
files.download(output_name+'.docx')
print('Now, please click the folder icon on the left-sidebar, right-click the file {} and select Download'.format(output_name+'.docx'))

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0

PackageNotFoundError: ignored